# Code to extract the data from the  ffc website

## Import the data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import lxml 

In [2]:
# need to manually save the file as csv from the website : http://www.f2concept.com/cal_aura/FR/

# load data from csv file, the first row is the header
df = pd.read_csv('Calendrier_aura.csv', sep=';', header=0, encoding='ISO-8859-1')

In [3]:
df['Catégorie'].unique()

array(['Gentlemen', 'Grimpée chronométrée', 'Test chronométré',
       'Ecole de route', 'Cyclosportive', 'Randonnée', 'Access hommes',
       'U17 hommes', 'Open1 hommes et +', 'Elites hommes (régional) et +',
       'Open2 hommes et +', 'Elites femmes (régional) et +',
       'U15+U17 femmes', 'U15 hommes', 'Elite nationale hommes',
       'Open3 hommes et +', 'U19 hommes fédéral', 'UCI Femmes',
       'U15 femmes', 'U17 femmes', 'U19 hommes régional', 'UCI U23',
       'UCI classe 1 et World Tour', 'U19 femmes régional', 'UCI Juniors',
       'Ultra distance', 'UCI classe 2'], dtype=object)

In [4]:
# change column names lieu to Lieu 
df.rename(columns={'lieu': 'Lieu'}, inplace=True)

# Keep only the following Catégoie : [Gentlemen, Grimpée chronométrée, Test chronométré, Access hommes]
courses_ffc = df[df['Catégorie'].isin(['Gentlemen', 'Grimpée chronométrée', 'Test chronométré', 'Access hommes', 'Cyclosportive'])]
courses_ffc['Du'] = courses_ffc['Du'].astype(str).str.replace('/23', '/2023')
courses_ffc['Au'] = courses_ffc['Au'].astype(str).str.replace('/23', '/2023')
courses_ffc.head(5)




C:\Users\theop\AppData\Local\Temp\ipykernel_12156\1637306300.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses_ffc['Du'] = courses_ffc['Du'].astype(str).str.replace('/23', '/2023')
C:\Users\theop\AppData\Local\Temp\ipykernel_12156\1637306300.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  courses_ffc['Au'] = courses_ffc['Au'].astype(str).str.replace('/23', '/2023')


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
0,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER
1,14/10/2023,14/10/2023,TROPHEE DES GRIMPEURS LAVOUTE CHILHAC FERRUSSAC,Grimpée chronométrée,VELO SPORT BRIVADOIS,Route,43,SAINT CIRGUES-LAVOUTE CHILHAC
2,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
3,14/10/2023,14/10/2023,CHALLENGE DES MURES,Test chronométré,E.C. MUROISE,Route,69,SAINT LAURENT DE MURE
5,08/10/2023,08/10/2023,CYCLO LES HÉROS,Cyclosportive,ALPES VELO,Route,1,SAINT-VULBAS


In [5]:
# Show the number of courses by region
courses_ffc['Département'].value_counts()

74    26
1     23
38    14
73     9
15     8
42     7
69     6
63     6
43     5
26     4
7      3
3      1
Name: Département, dtype: int64

## Extract valuable informations

In [6]:
# Split the datefram into 4 dataframes, one for each category
gentlemen = courses_ffc[courses_ffc['Catégorie'] == 'Gentlemen']
grimpée = courses_ffc[courses_ffc['Catégorie'] == 'Grimpée chronométrée']
chrono = courses_ffc[courses_ffc['Catégorie'] == 'Test chronométré']
access = courses_ffc[courses_ffc['Catégorie'] == 'Access hommes']

# save the number of rows for each dataframe
gentlemen_rows = gentlemen.shape[0]
grimpée_rows = grimpée.shape[0]
chrono_rows = chrono.shape[0]
access_rows = access.shape[0]



# change the date format to datetime
gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')
grimpée['Du'] = pd.to_datetime(grimpée['Du'], format='%d/%m/%Y')
chrono['Du'] = pd.to_datetime(chrono['Du'], format='%d/%m/%Y')
access['Du'] = pd.to_datetime(access['Du'], format='%d/%m/%Y')


# Sort the dataframe by date
gentlemen = gentlemen.sort_values(by=['Du'])
grimpée = grimpée.sort_values(by=['Du'])
chrono = chrono.sort_values(by=['Du'])
access = access.sort_values(by=['Du'])

# reverse the date y/m/d to d/m/y
gentlemen['Du'] = gentlemen['Du'].dt.strftime('%d/%m/%Y')
grimpée['Du'] = grimpée['Du'].dt.strftime('%d/%m/%Y')
chrono['Du'] = chrono['Du'].dt.strftime('%d/%m/%Y')
access['Du'] = access['Du'].dt.strftime('%d/%m/%Y')


# Check if there is a loss of data: the number of rows should be the same
print('gentlemen : ', gentlemen.shape[0] == gentlemen_rows)
print('grimpée : ', grimpée.shape[0] == grimpée_rows)
print('chrono : ', chrono.shape[0] == chrono_rows)
print('access : ', access.shape[0] == access_rows)



gentlemen :  True
grimpée :  True
chrono :  True
access :  True


C:\Users\theop\AppData\Local\Temp\ipykernel_20508\390231604.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')
C:\Users\theop\AppData\Local\Temp\ipykernel_20508\390231604.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grimpée['Du'] = pd.to_datetime(grimpée['Du'], format='%d/%m/%Y')
C:\Users\theop\AppData\Local\Temp\ipykernel_20508\390231604.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [7]:
access.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
557,26/02/2023,26/02/2023,PROLOGUE VALLEE DU BEDAT,Access hommes,VELO SPORT GERZATOIS,Route,63,BLANZAT
550,05/03/2023,05/03/2023,PRIX AUTO PIECES AUVERGNE,Access hommes,V C COURNON D'AUVERGNE,Route,63,COURNON D'AUVERGNE
541,11/03/2023,11/03/2023,PRIX TARARE POPEY,Access hommes,A.C. TARARE/POPEY,Route,69,SAINT ROMAIN DE POPEY
537,12/03/2023,12/03/2023,PRIX DE LA ZONE INDUSTRIELLE DE CUSSET,Access hommes,AVENIR CYCLISTE CUSSET,Route,3,CUSSET
524,12/03/2023,12/03/2023,2EME PRIX DE SAINT MAURICE DE SAINT REMENS,Access hommes,VELO CLUB D'AMBERIEU,Route,1,SAINT MAURICE DE REMENS


In [8]:
gentlemen.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
20,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V C PAYS DE SAINT-FLOUR,Route,15,PAULHAC / VALUÉJOLS
6,07/10/2023,07/10/2023,GENTLEMEN DE TAPONAS,Gentlemen,U.C. BELLEVILLOISE,Route,69,BELLEVILLE
2,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
0,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER


In [9]:
grimpée.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
431,23/04/2023,23/04/2023,MONTEE CHRONOMETREE DE SIRAN,Grimpée chronométrée,A C V AURILLACOIS,Route,15,LAROQUEBROU
353,20/05/2023,20/05/2023,TROPHEE DU PUY MARY GRAND SITE 1ÈRE EPREUVE,Grimpée chronométrée,SPRINTER CLUB AURILLACOIS,Route,15,DIENNE / PUY MARY
346,21/05/2023,21/05/2023,TROPHEE DU PUY MARY GRAND SITE 2ÈME EPREUVE,Grimpée chronométrée,SPRINTER CLUB AURILLACOIS,Route,15,LE FALGOUX / LE PUY MARY
345,21/05/2023,21/05/2023,TROPHEE DU PUY MARY GRAND SITE 3ÈME EPREUVE,Grimpée chronométrée,SPRINTER CLUB AURILLACOIS,Route,15,MANDAILLES ST JULIEN / PUY MARY
336,27/05/2023,27/05/2023,GRIMPEE DES VOIRONS,Grimpée chronométrée,V.C. DOUVAINE,Route,74,BONS EN CHABLAIS


In [10]:
chrono.head(5)


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
468,15/04/2023,15/04/2023,CHAMPIONNATS AUVERGNE-RHÔNE-ALPES ROUTE CLM PA...,Test chronométré,TEAM CYCLISTE CHATEL-GUYON,Route,63,CHATEL GUYON
21,24/09/2023,24/09/2023,CHRONO DE L'OMS - CHAMPIONNAT DE HAUTE-SAVOIE ...,Test chronométré,U.C. THONONAISE,Route,74,THONON LES BAINS
3,14/10/2023,14/10/2023,CHALLENGE DES MURES,Test chronométré,E.C. MUROISE,Route,69,SAINT LAURENT DE MURE


In [11]:
import dataframe_image as dfi
#Save the dataframes as an image
dfi.export(gentlemen, 'gentlemen.png', table_conversion= 'matplotlib')
dfi.export(grimpée, 'grimpée.png', table_conversion= 'matplotlib')
dfi.export(chrono, 'chrono.png', table_conversion= 'matplotlib')
dfi.export(access, 'access.png', table_conversion= 'matplotlib')




In [12]:
# change the date format to datetime
gentlemen['Du'] = pd.to_datetime(gentlemen['Du'], format='%d/%m/%Y')


# Sort the dataframe by date
gentlemen = gentlemen.sort_values(by=['Du'])

# reverse the date y/m/d to d/m/y
gentlemen['Du'] = gentlemen['Du'].dt.strftime('%d/%m/%Y')
gentlemen.head(5)




,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
20,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V C PAYS DE SAINT-FLOUR,Route,15,PAULHAC / VALUÉJOLS
6,07/10/2023,07/10/2023,GENTLEMEN DE TAPONAS,Gentlemen,U.C. BELLEVILLOISE,Route,69,BELLEVILLE
2,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE
0,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER


In [13]:
pd.set_option('display.max_rows', None)
chrono.query('`Département` == 74')


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu
21,24/09/2023,24/09/2023,CHRONO DE L'OMS - CHAMPIONNAT DE HAUTE-SAVOIE ...,Test chronométré,U.C. THONONAISE,Route,74,THONON LES BAINS


## Create a map

In [6]:
# location
import geocoder
import geopy
from geopy.geocoders import Nominatim
from functools import partial

# map 
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

from tqdm import tqdm
# remove the warning
import warnings
warnings.filterwarnings('ignore')



In [7]:
# Use the initial dataframe :
courses_map = courses_ffc.copy()
courses_map['info_place'] = 'Auvergne-Rhône-Alpes'  + ', ' + courses_map['Département'].astype(str) + ', ' + courses_map['Lieu']
courses_map['info_course'] = courses_map['Catégorie'] + ' - ' + courses_map['Nom de la course'] + ' - ' + courses_map['Du']

# find localisation 
geolocator = Nominatim(user_agent="courses_ffc")
tqdm.pandas()
courses_map['Location'] = courses_map['info_place'].progress_apply(partial(geolocator.geocode, timeout=5, language='fr'))
courses_map['Point'] = courses_map['Location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

# verification
print('failed to geocode for', courses_map['Point'].isnull().sum(), 'race(s)')

# extract null rows and apply geocode again
null_rows = courses_map[courses_map['Point'].isnull()]
null_rows['info_place'] = 'Auvergne-Rhône-Alpes' + ', ' + null_rows['Lieu']
tqdm.pandas()
null_rows['Location'] = null_rows['info_place'].progress_apply(partial(geolocator.geocode, timeout=5, language='fr'))
null_rows['Point'] = null_rows['Location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)

# verification
print('failed to geocode again for', null_rows['Point'].isnull().sum(), 'race(s)')
# show race with no location :
null_rows[null_rows['Point'].isnull()].head(20)


100%|██████████| 112/112 [00:00<00:00, 112544.81it/s]


failed to geocode for 12 race(s)


100%|██████████| 12/12 [00:00<?, ?it/s]

failed to geocode again for 12 race(s)


,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu,info_place,info_course,Location,Point
1,14/10/2023,14/10/2023,TROPHEE DES GRIMPEURS LAVOUTE CHILHAC FERRUSSAC,Grimpée chronométrée,VELO SPORT BRIVADOIS,Route,43,SAINT CIRGUES-LAVOUTE CHILHAC,"Auvergne-Rhône-Alpes, SAINT CIRGUES-LAVOUTE CH...",Grimpée chronométrée - TROPHEE DES GRIMPEURS L...,None,None
20,24/09/2023,24/09/2023,GENTLEMEN,Gentlemen,V C PAYS DE SAINT-FLOUR,Route,15,PAULHAC / VALUÉJOLS,"Auvergne-Rhône-Alpes, PAULHAC / VALUÉJOLS",Gentlemen - GENTLEMEN - 24/09/2023,None,None
155,30/07/2023,30/07/2023,GRIMPEE DE LA RAMAZ - TROPHEE DES GRIMPEES DE ...,Grimpée chronométrée,Comité départemental Haute-Savoie FFC,Route,74,MIEUSSY HAUTE SAVOIE,"Auvergne-Rhône-Alpes, MIEUSSY HAUTE SAVOIE",Grimpée chronométrée - GRIMPEE DE LA RAMAZ - T...,None,None
166,23/07/2023,23/07/2023,LES HAUTS DU RUESSIUM,Access hommes,UNION CYCLISTE LE PUY EN VELAY,Route,43,BLANZAC / SAINT PAULIEN,"Auvergne-Rhône-Alpes, BLANZAC / SAINT PAULIEN",Access hommes - LES HAUTS DU RUESSIUM - 23/07/...,None,None
188,15/07/2023,15/07/2023,L'AINDINOISE,Cyclosportive,ALPES VELO,Route,1,PARC DES OISEAUX > GRAND-COLOMBIER,"Auvergne-Rhône-Alpes, PARC DES OISEAUX > GRAND...",Cyclosportive - L'AINDINOISE - 15/07/2023,None,None
199,09/07/2023,09/07/2023,LA MAURIENNE CLASSIQUE CYCLO,Cyclosportive,CHAMBERY CYCLISME ORGANISATION,Route,73,SAINT JULIEN MONT DENIS CROIX DE FER,"Auvergne-Rhône-Alpes, SAINT JULIEN MONT DENIS ...",Cyclosportive - LA MAURIENNE CLASSIQUE CYCLO -...,None,None
204,08/07/2023,08/07/2023,GRIMPEE DE CHAMROUSSE,Grimpée chronométrée,G.M.C. 38 (EYBENS FORMATION),Route,38,URIAGE LES BAINS - CHAMROUSSE,"Auvergne-Rhône-Alpes, URIAGE LES BAINS - CHAMR...",Grimpée chronométrée - GRIMPEE DE CHAMROUSSE -...,None,None
216,02/07/2023,02/07/2023,JPP NEUF DE COEUR,Cyclosportive,V.C. CLUSES SCIONZIER,Route,74,CLUSES LES CARROZ,"Auvergne-Rhône-Alpes, CLUSES LES CARROZ",Cyclosportive - JPP NEUF DE COEUR - 02/07/2023,None,None
256,18/06/2023,18/06/2023,PRIX DES 4 COMMUNES DE LA COMBE DU VAL,Access hommes,E.C. ARBENT-MARCHON,Route,1,VIEUX D IZENAVE,"Auvergne-Rhône-Alpes, VIEUX D IZENAVE",Access hommes - PRIX DES 4 COMMUNES DE LA COM...,None,None
329,28/05/2023,28/05/2023,GRAND PRIX DE SAINT CLAIR DE LA TOUR,Access hommes,EC SAINTE CLAIROISE,Route,38,"BELLEFONTAINE, ROUTE DE LA CHAPELLE DE LA TOUR","Auvergne-Rhône-Alpes, BELLEFONTAINE, ROUTE DE ...",Access hommes - GRAND PRIX DE SAINT CLAIR DE L...,None,None


In [8]:
# add manually the location for the problematic rows :

# To do


# delete the null rows for null_rows
null_rows = null_rows[null_rows['Point'].notnull()]

# replace the null rows with the new values
courses_map.loc[courses_map['Point'].isnull(), 'Point'] = null_rows['Point']
courses_map.loc[courses_map['Point'].isnull(), 'Location'] = null_rows['Location']

# add the latitude and longitude to the dataframe
#courses_map['Latitude'] = courses_map['Point'].apply(lambda x: x[0] if x else None)
#courses_map['Longitude'] = courses_map['Point'].apply(lambda x: x[1] if x else None)
courses_map[['Latitude', 'Longitude', 'Alt']] = pd.DataFrame(courses_map['Point'].tolist(), index=courses_map.index)
# delete Alt column
del courses_map['Alt']
courses_map.head(5)

,Du,Au,Nom de la course,Catégorie,Organisateur,Discipline,Département,Lieu,info_place,info_course,Location,Point,Latitude,Longitude
0,15/10/2023,15/10/2023,GENTLEMAN DE SCIENTRIER,Gentlemen,Comité départemental Haute-Savoie FFC,Route,74,SCIENTRIER,"Auvergne-Rhône-Alpes, 74, SCIENTRIER",Gentlemen - GENTLEMAN DE SCIENTRIER - 15/10/2023,"(Scientrier, Saint-Julien-en-Genevois, Haute-S...","(46.1188775, 6.3163228, 0.0)",46.118878,6.316323
1,14/10/2023,14/10/2023,TROPHEE DES GRIMPEURS LAVOUTE CHILHAC FERRUSSAC,Grimpée chronométrée,VELO SPORT BRIVADOIS,Route,43,SAINT CIRGUES-LAVOUTE CHILHAC,"Auvergne-Rhône-Alpes, 43, SAINT CIRGUES-LAVOUT...",Grimpée chronométrée - TROPHEE DES GRIMPEURS L...,NaN,NaN,NaN,NaN
2,14/10/2023,14/10/2023,GENTLEMAN CALADOIS,Gentlemen,V.C. VILLEFRANCHE BEAUJOLAIS,Route,69,GLEIZE,"Auvergne-Rhône-Alpes, 69, GLEIZE",Gentlemen - GENTLEMAN CALADOIS - 14/10/2023,"(Gleizé, Villefranche-sur-Saône, Rhône, Auverg...","(45.9888277, 4.6971455, 0.0)",45.988828,4.697146
3,14/10/2023,14/10/2023,CHALLENGE DES MURES,Test chronométré,E.C. MUROISE,Route,69,SAINT LAURENT DE MURE,"Auvergne-Rhône-Alpes, 69, SAINT LAURENT DE MURE",Test chronométré - CHALLENGE DES MURES - 14/10...,"(Saint-Laurent-de-Mure, Lyon, Rhône, Auvergne-...","(45.6872816, 5.0461188, 0.0)",45.687282,5.046119
5,08/10/2023,08/10/2023,CYCLO LES HÉROS,Cyclosportive,ALPES VELO,Route,1,SAINT-VULBAS,"Auvergne-Rhône-Alpes, 1, SAINT-VULBAS",Cyclosportive - CYCLO LES HÉROS - 08/10/2023,"(Saint-Vulbas, Belley, Ain, Auvergne-Rhône-Alp...","(45.8306, 5.28959, 0.0)",45.830600,5.289590


In [9]:
# drop null rows
courses_map = courses_map[courses_map['Point'].notnull()]

In [10]:
# slightly change the latitude and longitude to avoid overlapping of the markers
courses_map['Latitude'] = courses_map['Latitude'] + np.random.uniform(-0.01, 0.01, courses_map.shape[0])
courses_map['Longitude'] = courses_map['Longitude'] + np.random.uniform(-0.01, 0.01, courses_map.shape[0])


In [141]:

# create a map with the courses location and marker color by category
map_courses = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')

##### MARKERS #####
for i in range(0,len(courses_map)):
    # change the marker icon by category
    if courses_map.iloc[i]['Catégorie'] == 'Gentlemen':
        folium.Marker([courses_map.iloc[i]['Latitude'], courses_map.iloc[i]['Longitude']], popup=courses_map.iloc[i]['info_course'], icon=folium.Icon(color='blue', prefix='fa', icon='bicycle')).add_to(map_courses)
    elif courses_map.iloc[i]['Catégorie'] == 'Grimpée chronométrée':
        folium.Marker([courses_map.iloc[i]['Latitude'], courses_map.iloc[i]['Longitude']], popup=courses_map.iloc[i]['info_course'], icon=folium.Icon(color='green', prefix='fa', icon='bicycle')).add_to(map_courses)
    elif courses_map.iloc[i]['Catégorie'] == 'Test chronométré':
        folium.Marker([courses_map.iloc[i]['Latitude'], courses_map.iloc[i]['Longitude']], popup=courses_map.iloc[i]['info_course'], icon=folium.Icon(color='orange', prefix='fa', icon='bicycle')).add_to(map_courses)
    elif courses_map.iloc[i]['Catégorie'] == 'Access hommes':
        folium.Marker([courses_map.iloc[i]['Latitude'], courses_map.iloc[i]['Longitude']], popup=courses_map.iloc[i]['info_course'], icon=folium.Icon(color='purple', prefix='fa', icon='bicycle')).add_to(map_courses)
    else:
        folium.Marker([courses_map.iloc[i]['Latitude'], courses_map.iloc[i]['Longitude']], popup=courses_map.iloc[i]['info_course'], icon=folium.Icon(color='red', prefix='fa', icon='bicycle')).add_to(map_courses)


##### LAYERS #####
# add button to switch between all the map types
folium.TileLayer('OpenStreetMap').add_to(map_courses)
folium.TileLayer('Stamen Terrain').add_to(map_courses)
# add a layer control
folium.LayerControl().add_to(map_courses)



##### LEGEND #####
# add a legend outside the map
legend_html =   '''
                <div style="position: fixed;
                            background-color:white; opacity:0.75;
                            bottom: 20px; right: 20px; width: 180px; height: 200px;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Légende : <br>
                                &nbsp; Gentlemen &nbsp; <i class="fa fa-bicycle fa-2x" style="color:blue"></i><br>
                                &nbsp; Grimpée &nbsp; <i class="fa fa-bicycle fa-2x" style="color:green"></i><br>
                                &nbsp; Test chronométré &nbsp; <i class="fa fa-bicycle fa-2x" style="color:orange"></i><br>
                                &nbsp; Access &nbsp; <i class="fa fa-bicycle fa-2x" style="color:purple"></i><br>
                                &nbsp; Cyclosportive &nbsp; <i class="fa fa-bicycle fa-2x" style="color:red"></i>
                </div>
                '''
# decrease the size of the legend
legend_html = legend_html.replace('font-size:1px', 'font-size:6px')
# write 'Légende' in bold
legend_html = legend_html.replace('Légende :', '<b>Légende :</b>')
map_courses.get_root().html.add_child(folium.Element(legend_html))







# save the map
map_courses.save('map_courses.html')

map_courses


In [40]:
# create a map with the courses location and marker color by category
map_courses = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')



##### MARKERS #####
# color by category : blue,       green,    orange,   purple,       red : 
#                   Gentlemen,   Grimpée,   Chrono,   Access,   Cyclosportive


# define dataframes by category :
gentlemen = courses_map[courses_map['Catégorie'] == 'Gentlemen']
grimpée = courses_map[courses_map['Catégorie'] == 'Grimpée chronométrée']
test = courses_map[courses_map['Catégorie'] == 'Test chronométré']
access = courses_map[courses_map['Catégorie'] == 'Access hommes']
cyclosportive = courses_map[courses_map['Catégorie'] == 'Cyclosportive']


##### LAYERS #####
# each layer corespond to a category
access_group = folium.FeatureGroup(name="Access").add_to(map_courses)
for i in range(0,len(access)):
    access_group.add_child(folium.Marker([access.iloc[i]['Latitude'], access.iloc[i]['Longitude']], popup=access.iloc[i]['info_course'], icon=folium.Icon(color='purple', prefix='fa', icon='bicycle')))
gentlemen_group = folium.FeatureGroup(name="Gentlemen").add_to(map_courses)
for i in range(0,len(gentlemen)):
    gentlemen_group.add_child(folium.Marker([gentlemen.iloc[i]['Latitude'], gentlemen.iloc[i]['Longitude']], popup=gentlemen.iloc[i]['info_course'], icon=folium.Icon(color='blue', prefix='fa', icon='bicycle')))
grimpée_group = folium.FeatureGroup(name="Grimpée").add_to(map_courses)
for i in range(0,len(grimpée)):
    grimpée_group.add_child(folium.Marker([grimpée.iloc[i]['Latitude'], grimpée.iloc[i]['Longitude']], popup=grimpée.iloc[i]['info_course'], icon=folium.Icon(color='green', prefix='fa', icon='bicycle')))
test_group = folium.FeatureGroup(name="Test chronométré").add_to(map_courses)
for i in range(0,len(test)):
    test_group.add_child(folium.Marker([test.iloc[i]['Latitude'], test.iloc[i]['Longitude']], popup=test.iloc[i]['info_course'], icon=folium.Icon(color='orange', prefix='fa', icon='bicycle')))
cyclosportive_group = folium.FeatureGroup(name="Cyclosportive").add_to(map_courses)
for i in range(0,len(cyclosportive)):
    cyclosportive_group.add_child(folium.Marker([cyclosportive.iloc[i]['Latitude'], cyclosportive.iloc[i]['Longitude']], popup=cyclosportive.iloc[i]['info_course'], icon=folium.Icon(color='red', prefix='fa', icon='bicycle')))

# add button to switch between OPENSTREETMAP and STAMEN TERRAIN
folium.TileLayer('OpenStreetMap').add_to(map_courses)
folium.TileLayer('Stamen Terrain').add_to(map_courses)
# add a layer control
folium.LayerControl().add_to(map_courses)




##### LEGEND #####
# add a legend outside the map
legend_html =   '''
                <div style="position: fixed;
                            background-color:white; opacity:0.75;
                            bottom: 20px; right: 20px; width: 180px; height: 200px;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Légende : <br>
                                &nbsp; Gentlemen &nbsp; <i class="fa fa-bicycle fa-2x" style="color:blue"></i><br>
                                &nbsp; Grimpée &nbsp; <i class="fa fa-bicycle fa-2x" style="color:green"></i><br>
                                &nbsp; Test chronométré &nbsp; <i class="fa fa-bicycle fa-2x" style="color:orange"></i><br>
                                &nbsp; Access &nbsp; <i class="fa fa-bicycle fa-2x" style="color:purple"></i><br>
                                &nbsp; Cyclosportive &nbsp; <i class="fa fa-bicycle fa-2x" style="color:red"></i>
                </div>
                '''
# decrease the size of the legend
legend_html = legend_html.replace('font-size:1px', 'font-size:6px')
# write 'Légende' in bold
legend_html = legend_html.replace('Légende :', '<b>Légende :</b>')
map_courses.get_root().html.add_child(folium.Element(legend_html))







# save the map
map_courses.save('map_courses.html')

map_courses


In [48]:
# create a map with the courses location and marker color by category
map_courses = folium.Map(location=[46, 6], zoom_start=8, tiles=' OpenStreetMap')



##### MARKERS #####
# color by category : blue,       green,    orange,   purple,       red : 
#                   Gentlemen,   Grimpée,   Chrono,   Access,   Cyclosportive


# define dataframes by category :
gentlemen = courses_map[courses_map['Catégorie'] == 'Gentlemen']
grimpée = courses_map[courses_map['Catégorie'] == 'Grimpée chronométrée']
test = courses_map[courses_map['Catégorie'] == 'Test chronométré']
access = courses_map[courses_map['Catégorie'] == 'Access hommes']
cyclosportive = courses_map[courses_map['Catégorie'] == 'Cyclosportive']


##### LAYERS #####

# add button to switch between OPENSTREETMAP and STAMEN TERRAIN
folium.TileLayer('OpenStreetMap').add_to(map_courses)
folium.TileLayer('Stamen Terrain').add_to(map_courses)
# add a layer control
folium.LayerControl().add_to(map_courses)



##### TimeLine #####
# create a timeline
timeline = folium.plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': [
            {
                'type': 'Feature',
                'geometry': {
                    'type': 'Point',
                    'coordinates': [gentlemen.iloc[i]['Longitude'], gentlemen.iloc[i]['Latitude']]
                },
                'properties': {
                    'times': gentlemen.iloc[i]['Du'],
                    'style': {'color': 'blue'},
                    'icon' : 'home',
                    'iconstyle': {
                        'color': 'blue',
                        'fillColor': 'blue',
                        'fillOpacity': 0.5,
                        'stroke': 'true',
                        'radius': 5
                    },
                    'popup': gentlemen.iloc[i]['info_course']
                }
            } for i in range(0,len(gentlemen))
        ]
    },

    period='P1M',
    add_last_point=True,
    auto_play=True,
    date_options='DD/MM/YYYY',
    loop=True,
    max_speed=1,
    loop_button=True,

    time_slider_drag_update=True
)

# add the timeline to the map
timeline.add_to(map_courses)







##### LEGEND #####
# add a legend outside the map
legend_html =   '''
                <div style="position: fixed;
                            background-color:white; opacity:0.75;
                            bottom: 20px; right: 20px; width: 180px; height: 200px;
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp; Légende : <br>
                                &nbsp; Gentlemen &nbsp; <i class="fa fa-bicycle fa-2x" style="color:blue"></i><br>
                                &nbsp; Grimpée &nbsp; <i class="fa fa-bicycle fa-2x" style="color:green"></i><br>
                                &nbsp; Test chronométré &nbsp; <i class="fa fa-bicycle fa-2x" style="color:orange"></i><br>
                                &nbsp; Access &nbsp; <i class="fa fa-bicycle fa-2x" style="color:purple"></i><br>
                                &nbsp; Cyclosportive &nbsp; <i class="fa fa-bicycle fa-2x" style="color:red"></i>
                </div>
                '''
# decrease the size of the legend
legend_html = legend_html.replace('font-size:1px', 'font-size:6px')
# write 'Légende' in bold
legend_html = legend_html.replace('Légende :', '<b>Légende :</b>')
map_courses.get_root().html.add_child(folium.Element(legend_html))







# save the map
map_courses.save('map_courses.html')

map_courses
